# Banane Group Creator 

### Packages

In [1]:
import json
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_bipartite_matching

### Constant

In [35]:
DAYS = {'MONDAY':0,'TUESDAY':1,'WEDNESDAY':2,'THURSDAY':3,'FRIDAY':4}
GROUP_SIZE = 5
TEMPERATURE = 100

### Data

In [36]:
preferences = {'Maria':{'MONDAY','TUESDAY'},'Boris':{'WEDNESDAY'},
              'Jeremy':{'TUESDAY'},'Farah':{'WEDNESDAY','THURSDAY'},
              'Gabriel':{'WEDNESDAY','TUESDAY'},'Alexia':{'THURSDAY'},
              'Ines':{'THURSDAY','TUESDAY'},'Shayan':{'WEDNESDAY','FRIDAY'},
              'Noah':{'MONDAY','TUESDAY','FRIDAY'},'Mariella':{'WEDNESDAY','MONDAY'},
              'Antoine':{'MONDAY','WEDNESDAY'},'Etienne':{'WEDNESDAY'},
              'Valentine':{'THURSDAY','TUESDAY'},'Bastien':{'WEDNESDAY','MONDAY','THURSDAY','FRIDAY'},
              'Benoit':{'MONDAY','TUESDAY'},'Billibob':{'MONDAY'},
              'Juliette':{'MONDAY','FRIDAY'},'Flavia':{'WEDNESDAY','MONDAY'},
              'Amelie':{'TUESDAY','FRIDAY','WEDNESDAY'},'Gaetan':{'WEDNESDAY','MONDAY','TUESDAY'},}

In [37]:
prior = {'Juliette':{'Benoit','Antoine'},'Benoit':{'Juliette','Antoine'},
         'Antoine':{'Benoit','Juliette'}}

Data Format:
- adjacency matrix NxM. 
- N is the number of people
- M is the number of slots (#days*group_size) (ordered)

In [38]:
people_mapping = dict(zip(preferences.keys(), range(len(preferences))))
day_mapping = [d+f'_{i+1}' for d in DAYS.keys() for i in range(GROUP_SIZE)]
day_mapping = dict(zip(day_mapping,range(len(day_mapping))))
adjacency_matrix = np.zeros((len(people_mapping),len(day_mapping)),dtype=np.float32)

### Code

#### Create weighted edges

In [39]:
weights = dict()
for name, days in preferences.items():
    for d in days:
        for i in range(GROUP_SIZE):
            weights[(name,d+f'_{i+1}')] = 1/len(days)

In [40]:
for name, friends in prior.items():
    overlap_days = set().union(preferences[name])
    for f in friends:
        overlap_days = overlap_days.intersection(preferences[f])
    if len(overlap_days) == 0:
        raise Exception(f'Impossible to match friends preferences for {name} as no overlapping days.')
    for d in list(overlap_days):
        for i in range(GROUP_SIZE):
            weights[(name,d+f'_{i+1}')] = weights[(name,d+f'_{i+1}')] + TEMPERATURE/len(overlap_days)

#### Convert to pandas DataFrame

In [41]:
weighted_preferences = [[*a,b] for a,b in weights.items()]

In [42]:
weighted_preferences_df = pd.DataFrame(weighted_preferences,columns=['name','day','weight'])
weighted_preferences_df.head()

,name,day,weight
0,Maria,MONDAY_1,0.5
1,Maria,MONDAY_2,0.5
2,Maria,MONDAY_3,0.5
3,Maria,MONDAY_4,0.5
4,Maria,MONDAY_5,0.5


#### Populate adjacency matrix

In [43]:
adjacency_matrix[weighted_preferences_df['name'].apply(lambda p: people_mapping[p]).to_list(),
                 weighted_preferences_df['day'].apply(lambda p: day_mapping[p]).to_list(
                 )] = weighted_preferences_df['weight'].to_list()

In [45]:
adjacency_matrix = csr_matrix(adjacency_matrix)

In [47]:
print(maximum_bipartite_matching(adjacency_matrix, perm_type='column'))

[ 0 10  5 11  6 15  7 22  1  2  3 13  8 16  9  4 20 14 21 12]


In [23]:
print(maximum_bipartite_matching(adjacency_matrix, perm_type='row'))

[ 0  8  9 10 15  2  4  6 12 14  1  3 19 11 17  5 13 -1 -1 -1 16 18  7 -1
 -1]


#### Create Graph and Matching

In [133]:
G = nx.from_pandas_edgelist(available_slots_df,source='name',target='day',
                            edge_attr='weight',create_using=nx.MultiGraph)
matching = nx.max_weight_matching(G,maxcardinality=True)

#### Format Matching

In [134]:
final_matching = dict()
for a,b in list(matching):
    if '_' in a:
        day, name = a.split('_')[0], b
    else:
        day, name = b.split('_')[0], a
    if day not in final_matching:
        final_matching[day] = []
    final_matching[day].append(name)
for day in final_matching.keys():
    final_matching[day] = sorted(final_matching[day])
    people = final_matching[day]
    if len(people) < GROUP_SIZE:
        people.extend(['']*(GROUP_SIZE-len(people)))

In [135]:
final_matching_df = pd.DataFrame(final_matching).T
final_matching_df = final_matching_df.sort_index(key=lambda idx: [DAYS[i] for i in idx])

### Results

In [136]:
final_matching_df

,0,1,2,3,4
MONDAY,Billibob,Flavia,Mariella,,
TUESDAY,Benoit,Jeremy,Maria,,
WEDNESDAY,Antoine,Boris,Etienne,Gabriel,Gaetan
THURSDAY,Alexia,Farah,Ines,Valentine,
FRIDAY,Amelie,Bastien,Juliette,Noah,Shayan


---

In [117]:
available_slots = []
for name, days in preferences.items():
    for i in range(GROUP_SIZE):
        available_slots.extend(zip([name]*len(days),
                                   [d+f'_{i+1}' for d in list(days)],
                                    [1/len(days)]*len(days)))
for name, friends in prior.items():
    overlap_days = set().union(preferences[name])
    for f in friends:
        overlap_days = overlap_days.intersection(preferences[f])
    if len(overlap_days) == 0:
        raise Exception(f'Impossible to match friends preferences for {name} as no overlapping days.')
    for d in list(overlap_days):
        for i in range(GROUP_SIZE):
            available_slots.extend(zip([name]*len(overlap_days),
                                       [d+f'_{i+1}' for d in list(overlap_days)],
                                        [TEMPERATURE/len(overlap_days)]*len(overlap_days)))